In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 🚕 Projet Taxi NYC - Analyse Comparative
# MAGIC ## Statistiques Inférentielles vs Big Data
# MAGIC 
# MAGIC **Entreprise:** DATACO  
# MAGIC **Période:** 2022-2025  
# MAGIC **Binôme:** [Vos noms]  
# MAGIC **Date:** 26-30 janvier 2026

# COMMAND ----------

# MAGIC %md
# MAGIC ## 📦 1. Configuration & Imports

# COMMAND ----------

# Imports PySpark
from pyspark.sql.functions import (
    col, count, mean, stddev, sum as spark_sum, min as spark_min, max as spark_max,
    hour, dayofweek, dayofmonth, month, year, weekofyear,
    unix_timestamp, percentile_approx, when, lit
)
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Imports Python scientifique
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import t, norm

# Imports visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# COMMAND ----------

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 📂 2. Chargement des Données

# COMMAND ----------

from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, LongType

# CHEMINS DES DONNÉES
PATH_POPULATION = "/Volumes/workspace/trips/population/"
PATH_SAMPLE = "/Volumes/workspace/trips/sample/"

# ============================================
# CHARGEMENT ÉCHANTILLON (CSV)
# ============================================
print("🔄 Chargement de l'échantillon...")
df_sample = spark.read.csv(
    PATH_SAMPLE + "yellowtaxisample1pct_hybrid_stratified.csv",
    header=True,
    inferSchema=True
)
nb_sample = df_sample.count()
print(f"✅ Échantillon chargé : {nb_sample:,} courses")

# ============================================
# CHARGEMENT POPULATION - Approche fichier par fichier
# ============================================
print("\n🔄 Chargement de la population complète...")

# Lister tous les fichiers parquet
files = [f.path for f in dbutils.fs.ls(PATH_POPULATION) if f.path.endswith('.parquet')]
print(f"📂 {len(files)} fichiers trouvés")

# Charger tous les fichiers avec conversion automatique
dfs = []
for i, file_path in enumerate(files, 1):
    try:
        # Lire le fichier
        df_temp = spark.read.parquet(file_path)
        
        # Convertir les colonnes qui peuvent être INT64 ou DOUBLE
        columns_to_convert = [
            "passenger_count", "trip_distance", "RatecodeID",
            "fare_amount", "extra", "mta_tax", "tip_amount", 
            "tolls_amount", "improvement_surcharge", "total_amount",
            "congestion_surcharge", "airport_fee"
        ]
        
        for col_name in columns_to_convert:
            if col_name in df_temp.columns:
                df_temp = df_temp.withColumn(col_name, col(col_name).cast(DoubleType()))
        
        dfs.append(df_temp)
        print(f"✅ [{i}/{len(files)}] {file_path.split('/')[-1]}")
        
    except Exception as e:
        print(f"⚠️ Erreur sur {file_path.split('/')[-1]}: {str(e)}")

# Fusionner tous les DataFrames
if dfs:
    from functools import reduce
    from pyspark.sql import DataFrame
    
    print("\n🔄 Fusion de tous les fichiers...")
    df_population = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dfs)
    
    nb_population = df_population.count()
    print(f"✅ Population totale chargée : {nb_population:,} courses")
else:
    raise Exception("❌ Aucun fichier n'a pu être chargé")

# Vérification ratio
ratio = (nb_sample / nb_population) * 100
print(f"\n📊 Ratio échantillon/population : {ratio:.2f}%")

# COMMAND ----------

# Vérification des schémas finaux
print("=== SCHÉMA POPULATION ===")
df_population.printSchema()

print("\n=== SCHÉMA ÉCHANTILLON ===")
df_sample.printSchema()

In [0]:


# MAGIC %md
# MAGIC ## 🔍 3. EDA - Exploration des Données

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.1 Structure des données

# COMMAND ----------

print("=== SCHÉMA POPULATION ===")
df_population.printSchema()

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.2 Aperçu des données

# COMMAND ----------

print("=== APERÇU POPULATION (5 premières lignes) ===")
display(df_population.limit(5))

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.3 Statistiques descriptives de base

# COMMAND ----------

# Population
print("=== STATISTIQUES DESCRIPTIVES - POPULATION ===")
stats_pop = df_population.select(
    "fare_amount", "trip_distance", "tip_amount", 
    "tolls_amount", "total_amount", "passenger_count"
).describe()
display(stats_pop)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.4 Vérification des valeurs manquantes

# COMMAND ----------
from pyspark.sql.functions import col, sum as spark_sum, isnan, when, count
# Fonction pour compter valeurs manquantes
def count_nulls(df, dataset_name):
    print(f"\n=== VALEURS MANQUANTES - {dataset_name} ===")
    null_counts = df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c) 
        for c in df.columns
    ])
    
    # Conversion en pandas pour affichage plus lisible
    null_df = null_counts.toPandas().T
    null_df.columns = ['Nb_Nulls']
    null_df['Pct_Nulls'] = (null_df['Nb_Nulls'] / df.count() * 100).round(2)
    null_df = null_df[null_df['Nb_Nulls'] > 0].sort_values('Nb_Nulls', ascending=False)
    
    if len(null_df) > 0:
        print(null_df)
    else:
        print("✅ Aucune valeur manquante détectée")
    
    return null_df

# Vérification
nulls_pop = count_nulls(df_population, "POPULATION")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.5 Détection préliminaire des outliers

# COMMAND ----------

# Calcul des quartiles pour fare_amount
quantiles_fare = df_population.approxQuantile("fare_amount", [0.01, 0.25, 0.50, 0.75, 0.99], 0.01)
Q1, Q3 = quantiles_fare[1], quantiles_fare[3]
IQR = Q3 - Q1

print("=== ANALYSE FARE_AMOUNT ===")
print(f"Q1 (25%): ${Q1:.2f}")
print(f"Médiane (50%): ${quantiles_fare[2]:.2f}")
print(f"Q3 (75%): ${Q3:.2f}")
print(f"IQR: ${IQR:.2f}")
print(f"1er percentile: ${quantiles_fare[0]:.2f}")
print(f"99e percentile: ${quantiles_fare[4]:.2f}")

# Limites outliers (méthode IQR)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(f"\nLimites outliers (IQR ±1.5):")
print(f"Limite inférieure: ${lower_bound:.2f}")
print(f"Limite supérieure: ${upper_bound:.2f}")

# Comptage outliers
nb_outliers = df_population.filter(
    (col("fare_amount") < lower_bound) | (col("fare_amount") > upper_bound)
).count()
pct_outliers = (nb_outliers / nb_population) * 100

print(f"\n📊 Outliers détectés: {nb_outliers:,} ({pct_outliers:.2f}%)")

# COMMAND ----------

In [0]:
# MAGIC %md
# MAGIC ## 💾 5. Analyse Big Data (Population Complète)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.1 Calcul des métriques EXACTES

# COMMAND ----------

import time
from pyspark.sql.functions import col, mean, unix_timestamp, count as spark_count, sum as spark_sum

# Mesure du temps de calcul
start_time = time.time()

print("=== CALCUL DES MÉTRIQUES EXACTES (POPULATION) ===")
print("🔄 Calcul en cours...")

# Prix moyen exact
mean_fare_exact = df_population.agg(mean("fare_amount")).collect()[0][0]
print(f"✅ Prix moyen EXACT: ${mean_fare_exact:.2f}")

# Distance moyenne exacte
mean_distance_exact = df_population.agg(mean("trip_distance")).collect()[0][0]
print(f"✅ Distance moyenne EXACTE: {mean_distance_exact:.2f} miles")

# Durée moyenne exacte
# ✅ CORRECTION : Utiliser unix_timestamp() au lieu de cast("long")
df_population_duration = df_population.withColumn(
    "duration_minutes",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)

# Vérification rapide
print("\n📊 Aperçu des durées calculées:")
df_population_duration.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "duration_minutes").show(5)

# Calcul de la moyenne exacte
mean_duration_exact = df_population_duration.agg(mean("duration_minutes")).collect()[0][0]
print(f"✅ Durée moyenne EXACTE: {mean_duration_exact:.2f} minutes")

# Proportion exacte avec tip
nb_with_tip_exact = df_population.filter(col("tip_amount") > 0).count()
prop_tip_exact = nb_with_tip_exact / nb_population
print(f"✅ Proportion EXACTE avec tip: {prop_tip_exact:.2%}")

# Temps de calcul
elapsed_time = time.time() - start_time
print(f"\n⏱️ Temps de calcul: {elapsed_time:.2f} secondes")

# Stockage pour comparaison
results_bigdata = {
    'mean_fare': mean_fare_exact,
    'mean_distance': mean_distance_exact,
    'mean_duration': mean_duration_exact,
    'prop_tip': prop_tip_exact,
    'compute_time': elapsed_time
}

print("\n✅ Toutes les métriques exactes calculées!")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.2 Distribution temporelle - Heures de pointe

# COMMAND ----------

from pyspark.sql.functions import hour, dayofweek, dayofmonth, month

# Ajout des colonnes temporelles + durée
df_population_time = df_population.withColumn(
    "duration_minutes",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
).withColumn("hour", hour("tpep_pickup_datetime")) \
 .withColumn("dayofweek", dayofweek("tpep_pickup_datetime")) \
 .withColumn("day", dayofmonth("tpep_pickup_datetime")) \
 .withColumn("month", month("tpep_pickup_datetime"))

# Analyse par heure
courses_par_heure = df_population_time.groupBy("hour").agg(
    count("*").alias("nb_courses"),
    mean("fare_amount").alias("fare_moyen"),
    mean("trip_distance").alias("distance_moyenne"),
    mean("duration_minutes").alias("duree_moyenne")
).orderBy("hour")

print("=== DISTRIBUTION PAR HEURE ===")
display(courses_par_heure)

# Identification heures de pointe
print("\n=== TOP 5 HEURES DE POINTE ===")
heures_pointe = courses_par_heure.orderBy(col("nb_courses").desc()).limit(5)
display(heures_pointe)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.3 Distribution par jour de la semaine

# COMMAND ----------

# Analyse par jour de semaine (1=Dimanche, 7=Samedi)
courses_par_jour = df_population_time.groupBy("dayofweek").agg(
    count("*").alias("nb_courses"),
    mean("fare_amount").alias("fare_moyen"),
    mean("duration_minutes").alias("duree_moyenne")
).orderBy("dayofweek")

# Mapping des jours (pour référence)
jours_mapping = {
    1: "Dimanche", 2: "Lundi", 3: "Mardi", 4: "Mercredi",
    5: "Jeudi", 6: "Vendredi", 7: "Samedi"
}

print("=== DISTRIBUTION PAR JOUR DE LA SEMAINE ===")
print("(1=Dimanche, 2=Lundi, 3=Mardi, 4=Mercredi, 5=Jeudi, 6=Vendredi, 7=Samedi)")
display(courses_par_jour)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.4 Analyse géographique par boroughs

# COMMAND ----------

# Vérifier si les colonnes borough existent
if "pickup_borough" in df_population.columns and "dropoff_borough" in df_population.columns:
    
    # Analyse complète par borough
    analyse_boroughs = df_population.groupBy("pickup_borough").agg(
        count("*").alias("nb_courses"),
        mean("fare_amount").alias("fare_moyen"),
        mean("trip_distance").alias("distance_moyenne"),
        mean("tip_amount").alias("tip_moyen"),
        (mean("tip_amount") / mean("fare_amount") * 100).alias("tip_pct_fare")
    ).orderBy(col("nb_courses").desc())

    print("=== ANALYSE PAR BOROUGH (PICKUP) ===")
    display(analyse_boroughs)

    # Statistiques par paire origine-destination
    top_routes = df_population.groupBy("pickup_borough", "dropoff_borough").agg(
        count("*").alias("nb_courses"),
        mean("fare_amount").alias("fare_moyen"),
        mean("trip_distance").alias("distance_moyenne")
    ).orderBy(col("nb_courses").desc()).limit(10)

    print("\n=== TOP 10 ROUTES (ORIGINE → DESTINATION) ===")
    display(top_routes)
    
else:
    print("⚠️ Les colonnes 'pickup_borough' et 'dropoff_borough' ne sont pas présentes dans les données")
    print("📍 Analyse par zones (PULocationID / DOLocationID) à la place:")
    
    # Analyse par zones de pickup
    analyse_zones = df_population.groupBy("PULocationID").agg(
        count("*").alias("nb_courses"),
        mean("fare_amount").alias("fare_moyen"),
        mean("trip_distance").alias("distance_moyenne"),
        mean("tip_amount").alias("tip_moyen")
    ).orderBy(col("nb_courses").desc()).limit(10)
    
    print("\n=== TOP 10 ZONES DE PICKUP ===")
    display(analyse_zones)
    
    # Top routes par zones
    top_routes_zones = df_population.groupBy("PULocationID", "DOLocationID").agg(
        count("*").alias("nb_courses"),
        mean("fare_amount").alias("fare_moyen"),
        mean("trip_distance").alias("distance_moyenne")
    ).orderBy(col("nb_courses").desc()).limit(10)
    
    print("\n=== TOP 10 ROUTES (ZONES) ===")
    display(top_routes_zones)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.5 Détection et analyse des outliers

# COMMAND ----------

# Calcul des percentiles pour plusieurs variables
print("=== ANALYSE DES OUTLIERS ===")

# Fare amount
quantiles_fare_full = df_population.approxQuantile("fare_amount", [0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99], 0.01)
Q1_fare, Q3_fare = quantiles_fare_full[2], quantiles_fare_full[4]
IQR_fare = Q3_fare - Q1_fare
lower_bound_fare = Q1_fare - 1.5 * IQR_fare
upper_bound_fare = Q3_fare + 1.5 * IQR_fare

print(f"\n--- FARE_AMOUNT ---")
print(f"1er percentile: ${quantiles_fare_full[0]:.2f}")
print(f"5e percentile: ${quantiles_fare_full[1]:.2f}")
print(f"Q1 (25e percentile): ${quantiles_fare_full[2]:.2f}")
print(f"Médiane (50e percentile): ${quantiles_fare_full[3]:.2f}")
print(f"Q3 (75e percentile): ${quantiles_fare_full[4]:.2f}")
print(f"95e percentile: ${quantiles_fare_full[5]:.2f}")
print(f"99e percentile: ${quantiles_fare_full[6]:.2f}")
print(f"\nIQR: ${IQR_fare:.2f}")
print(f"Limites IQR (±1.5×IQR): [${lower_bound_fare:.2f}, ${upper_bound_fare:.2f}]")

# Comptage outliers
outliers_fare = df_population.filter(
    (col("fare_amount") < lower_bound_fare) | (col("fare_amount") > upper_bound_fare)
)
nb_outliers_fare = outliers_fare.count()
pct_outliers_fare = (nb_outliers_fare / nb_population) * 100

print(f"\n📊 Outliers détectés: {nb_outliers_fare:,} ({pct_outliers_fare:.2f}%)")

# Analyse des outliers extrêmes (>99e percentile)
print("\n--- COURSES AVEC PRIX EXTRÊMES (>99e percentile) ---")

# Vérifier si les colonnes borough existent
if "pickup_borough" in df_population.columns:
    extremes_high = df_population.filter(col("fare_amount") > quantiles_fare_full[6]).select(
        "fare_amount", "trip_distance", "pickup_borough", "dropoff_borough"
    ).orderBy(col("fare_amount").desc()).limit(10)
else:
    extremes_high = df_population.filter(col("fare_amount") > quantiles_fare_full[6]).select(
        "fare_amount", "trip_distance", "PULocationID", "DOLocationID"
    ).orderBy(col("fare_amount").desc()).limit(10)

display(extremes_high)

# Statistiques des outliers
print("\n--- STATISTIQUES DES OUTLIERS ---")
outliers_stats = outliers_fare.select(
    mean("fare_amount").alias("fare_moyen_outliers"),
    mean("trip_distance").alias("distance_moyenne_outliers"),
    spark_min("fare_amount").alias("fare_min"),
    spark_max("fare_amount").alias("fare_max")
).collect()[0]

print(f"Prix moyen des outliers: ${outliers_stats['fare_moyen_outliers']:.2f}")
print(f"Distance moyenne des outliers: {outliers_stats['distance_moyenne_outliers']:.2f} miles")
print(f"Prix minimum: ${outliers_stats['fare_min']:.2f}")
print(f"Prix maximum: ${outliers_stats['fare_max']:.2f}")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 5.6 Ratio tip/fare par type de paiement (population)

# COMMAND ----------

from pyspark.sql.functions import stddev

# Calcul exact pour toute la population
df_population_ratio = df_population.filter(
    (col("fare_amount") > 0) & (col("tip_amount") >= 0)
).withColumn(
    "tip_ratio", col("tip_amount") / col("fare_amount")
)

ratio_by_payment_exact = df_population_ratio.groupBy("payment_type").agg(
    count("*").alias("nb_courses"),
    mean("tip_ratio").alias("ratio_tip_fare_moyen"),
    stddev("tip_ratio").alias("std_ratio"),
    mean("tip_amount").alias("tip_moyen"),
    mean("fare_amount").alias("fare_moyen")
).orderBy(col("nb_courses").desc())

print("=== RATIO TIP/FARE PAR TYPE DE PAIEMENT - POPULATION EXACTE ===")
print("Note: 1=Carte de crédit, 2=Cash, 3=No charge, 4=Dispute, 5=Unknown, 6=Voided trip")
display(ratio_by_payment_exact)

# Analyse détaillée
print("\n=== ANALYSE DÉTAILLÉE ===")
for row in ratio_by_payment_exact.collect():
    payment_type = row['payment_type']
    nb = row['nb_courses']
    ratio = row['ratio_tip_fare_moyen']
    tip_moyen = row['tip_moyen']
    fare_moyen = row['fare_moyen']
    
    print(f"\nType de paiement {payment_type}:")
    print(f"  • Nombre de courses: {nb:,}")
    print(f"  • Ratio tip/fare: {ratio:.2%}")
    print(f"  • Tip moyen: ${tip_moyen:.2f}")
    print(f"  • Fare moyen: ${fare_moyen:.2f}")

# COMMAND ----------